## Expired Futures
When searching for an expired asset, you will need to specify the date range based on the ExpiryDate property.

#### NuGet Packages

In [1]:
#r "nuget:Refinitiv.DataPlatform.Content, 1.0.0-alpha3"
#r "nuget:Microsoft.Data.Analysis"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed package Microsoft.Data.Analysis version 0.4.0

Installed package Refinitiv.DataPlatform.Content version 1.0.0-alpha3

In [2]:
using Newtonsoft.Json.Linq;
using Refinitiv.DataPlatform.Content.SearchService;
using Refinitiv.DataPlatform.Core;
using Microsoft.Data.Analysis;
using Microsoft.AspNetCore.Html;
using System.Linq;
using System;

#### Table Output
Helper routine to output data in a table format.

In [3]:
Formatter.Register<IList<JObject>>((hits, writer) =>
{
    if (hits.Count > 0)
    {
        var fields = new List<String>();
        var rows = new List<ICollection<IHtmlContent>>();
        var data = new Dictionary<string, IHtmlContent>();

        foreach(var hit in hits)
        {
            var cells = new List<IHtmlContent>();
            foreach (var val in hit.Properties())
            {
                if ( !fields.Any(item => item.Equals(val.Name)) )
                    fields.Add(val.Name);
                data[val.Name] = td(val.Value.ToString());
            }
            rows.Add(new List<IHtmlContent>(data.Values));
            data.Keys.ToList().ForEach(x => data[x] = td(""));
        }
        
        var headers = new List<IHtmlContent>();
        headers.AddRange(fields.Select(c => (IHtmlContent)th(c)));

        var t = table(thead(headers), tbody(rows.Select(r => tr(r))));
        writer.Write(t);
    }
}, "text/html");

In [5]:
// Create a session into the desktop
var session = CoreFactory.CreateSession(new DesktopSession.Params()
                            .AppKey("Your API Key here")
                            .OnState((s, state, msg) => Console.WriteLine($"{DateTime.Now}:{msg}. (State: {state})"))
                            .OnEvent((s, eventCode, msg) => Console.WriteLine($"{DateTime.Now}:{msg}. (Event: {eventCode})")));
session.Open();

17/12/2020 12:18:51 PM:Session is Pending. (State: Pending)
17/12/2020 12:18:52 PM:{
  "Contents": "Desktop Session Successfully Authenticated"
}. (Event: SessionAuthenticationSuccess)
17/12/2020 12:18:52 PM:Session is Opened. (State: Opened)


#### Example - Expired Vodafone futures in the month of Nov 2019.

In [8]:
var filter = "TickerSymbol eq 'vod' and ExpiryDate lt 2019-12-01 and ExpiryDate ge 2019-11-01";
var select = "DTSubjectName, ExchangeName, RIC, RCSAssetCategoryLeaf, AssetState, ExpiryDate, UnderlyingQuoteRIC";

var response = Search.Definition(Search.View.DerivativeQuotes).Filter(filter)
                                                              .Select(select)
                                                              .GetData();
response.Data.Hits

Johannesburg Stock Exchange Vodacom Group SSF Equity Future Nov 2019,Johannesburg Stock Exchange,VODQX9^1,Equity Future,DC,21/11/2019 12:00:00 AM,"[ ""VODJ.J"" ]"
Johannesburg Stock Exchange Vodacom Group Limited Cash Settled SSF Equity Future Nov 2019,Johannesburg Stock Exchange,VODSX9^1,Equity Future,DC,21/11/2019 12:00:00 AM,"[ ""VODJ.J"" ]"


#### Example - Expired Crude Oil Futures in the last year

In [9]:
// Build expiry date range expression
var dateOneYearAgo = DateTime.Now.AddYears(-1);
var dateRange = $"ExpiryDate ge {dateOneYearAgo:yyyy-MM-dd} and ExpiryDate lt {DateTime.Now:yyyy-MM-dd}";

In [10]:
filter = $@"RCSUnderlyingProductName xeq 'Energy' and
            SearchAllCategoryv3 eq 'Commodities' and
            RCSAssetCategoryName eq 'Derivative\Future' and RCSUnderlyingProductName eq 'Crude Oil' and
            ExchangeName xeq 'Source is a Refinitiv Contributor' and {dateRange}";

response = Search.Definition(Search.View.SearchAll).Filter(filter)
                                                   .OrderBy("ExpiryDate desc")
                                                   .Top(30)
                                                   .Select("DTSubjectName, RIC, ExpiryDate, RCSUnderlyingProductLeaf")
                                                   .GetData();
response.Data.Hits

NYM WTI Crude Implied Volatility Surface Delta ATM Month Jan 2021,CLATMIVMF21,16/12/2020 12:00:00 AM,Crude Oil
NYM WTI Crude Implied Volatility Surface Delta 15 Call Month Jan 2021,CL15CIVMF21,16/12/2020 12:00:00 AM,Crude Oil
NYM WTI Crude Implied Volatility Surface Delta 25 Put Month Jan 2021,CL25PIVMF21,16/12/2020 12:00:00 AM,Crude Oil
NYM WTI Crude Implied Volatility Surface Delta 15 Put Month Jan 2021,CL15PIVMF21,16/12/2020 12:00:00 AM,Crude Oil
NYM WTI Crude Implied Volatility Surface Delta 35 Put Month Jan 2021,CL35PIVMF21,16/12/2020 12:00:00 AM,Crude Oil
NYM WTI Crude Implied Volatility Surface Delta 35 Call Month Jan 2021,CL35CIVMF21,16/12/2020 12:00:00 AM,Crude Oil
NYM WTI Crude Implied Volatility Surface Delta 25 Call Month Jan 2021,CL25CIVMF21,16/12/2020 12:00:00 AM,Crude Oil
NYM WTI Crude Implied Volatility Surface Delta 40 Call Month Jan 2021,CL40CIVMF21,16/12/2020 12:00:00 AM,Crude Oil
NYM WTI Crude Implied Volatility Surface Delta 40 Put Month Jan 2021,CL40PIVMF21,16/12/2020 12:00:00 AM,Crude Oil
NYM WTI Crude Implied Volatility Surface Delta 10 Call Month Jan 2021,CL10CIVMF21,16/12/2020 12:00:00 AM,Crude Oil
NYM WTI Crude Implied Volatility Surface Delta 20 Put Month Jan 2021,CL20PIVMF21,16/12/2020 12:00:00 AM,Crude Oil


#### Example - Expired Electricity Futures in the last year

In [11]:
filter = $@"SearchAllCategoryv3 eq 'Commodities' and
            RCSUnderlyingProductName eq 'Power' and
            ExchangeName xeq 'Source is a Refinitiv Contributor' and {dateRange}";

response = Search.Definition(Search.View.SearchAll).Filter(filter)
                                                   .OrderBy("ExpiryDate desc")
                                                   .Top(30)
                                                   .Select("DTSubjectName, RIC, ExpiryDate, RCSUnderlyingProductLeaf")
                                                   .GetData();
response.Data.Hits

TRPC Electricity Germany Baseload Day 1,TRDEBD1,16/12/2020 12:00:00 AM,Baseload
TRPC Electricity France Baseload Day 1,TRFRBD1,16/12/2020 12:00:00 AM,Baseload
TRPC Electricity Germany Peakload Day 1,TRDEPD1,16/12/2020 12:00:00 AM,Peakload
TRPC Electricity France Peakload Day 1,TRFRPD1,16/12/2020 12:00:00 AM,Peakload
TRPC Electricity Netherlands Baseload Day 1,TRNLBD1,16/12/2020 12:00:00 AM,Baseload
TRPC Electricity Czech Republic Baseload Day 1,TRCZBD1,16/12/2020 12:00:00 AM,Baseload
TRPC Electricity Italy Baseload Day 1,TRITBD1,16/12/2020 12:00:00 AM,Baseload
TRPC Electricity Spain Baseload Day 1,TRESBD1,16/12/2020 12:00:00 AM,Baseload
TRPC Electricity Hungary Baseload Day 1,TRHUBD1,16/12/2020 12:00:00 AM,Baseload
TRPC Electricity Poland Baseload Day 1,TRPLBD1,16/12/2020 12:00:00 AM,Baseload
TRPC Electricity Czech Republic Peakload Day 1,TRCZPD1,16/12/2020 12:00:00 AM,Peakload
